In [2]:
import selenium
from selenium import webdriver
import os
import time

import urllib.request
import requests
from bs4 import BeautifulSoup

In [8]:
def create_soup(url):
    ''' create bs4 object '''
    
    r = requests.get(url, headers={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36"
    })
    return BeautifulSoup(r.content, "html5lib")

def next_page_url(url):
    ''' returns the url for the next page on an indeed job search page '''
    
    soup = create_soup(url)
    
    # this creates a list of "Results Page" at bottom of screen... last url will always be next page 
    next_pages_urls = []
    for i in soup.find_all(attrs={'class': 'pagination'}):
        a_tags = i.find_all('a')
        for a in a_tags:
            next_pages = 'https://www.indeed.com' + a['href']
            next_pages_urls.append(next_pages)

    return next_pages_urls[-1]


def scrape_job_links(url):
    
    soup = create_soup(url)
    
    # get hrefs ONLY for non- header and footer job postings 
    lst1= []
    for div in soup.find_all('div', {'class':' row result'}):
        links = div.find('a')['href']
        #print(links)
        lst1.append(links)

    # last row has different class name
    for div in soup.find_all('div', {'class':'lastRow row result'}):
        links = div.find('a')['href']
        #print(div)
        lst1.append(links)
    
    links = ['https://www.indeed.com' + i for i in lst1]
    return links

In [9]:
def bottom_scroll(webdriver):
    ''''scroll to bottom of page--- w/ page search pages -- not individual job postings'''
    element=webdriver.find_element_by_xpath('//*[@id="resultsCol"]/div[18]')
    return element.location_once_scrolled_into_view

In [12]:
def indeed_scraper(): 
    '''this combined everything '''
    

    driver = setup_webdriver()
    
    # set website to scrape and query 
    driver.get("https://www.indeed.com/")
    driver.find_element_by_id('what').clear()
    driver.find_element_by_id('what').send_keys('python developer')
    driver.find_element_by_id('where').clear()
    driver.find_element_by_id('where').send_keys('New York, NY')
    driver.find_element_by_id('fj').click()

    
    # x out advertisement 
    driver.find_element_by_id('prime-popover-x').click()
    
    # get current url and pass into bs4
    this_url = driver.current_url
    date_soup = create_soup(this_url)
    
    # sort by date 
    date_button = date_soup.find(attrs={'class': 'no-wrap'})
    date_button2 = date_button.find('a')['href']
    sort_by_date = 'https://www.indeed.com' + date_button2
    
    # pass url to driver & get current url to scrape links 
    driver.get(sort_by_date)
    
    # scrape em
    all_job_links = []
    for i in range(5):
        # get current url
        driver.get(driver.current_url)
        
        # get job links & append to list
        job_links = scrape_job_links(driver.current_url)
        all_job_links.extend(job_links)
        
        # scroll to the bottom of screen
        bottom_scroll(driver)
        
        # go to next page
        url_next = next_page_url(driver.current_url)
        driver.get(url_next)
    return all_job_links

In [13]:
jobs = indeed_scraper()

In [23]:
# need to figure out 
def filter_links(links_lst):
    
    
    indeed_links = []
    for i in links_lst:
        if i.startswith('https://www.indeed.com/rc'):
            pass
        else:
            #time.sleep(2)
            #webdriver_obj.get(i) #!!!!!! CHANGE THIS
            indeed_links.append(i)
    return indeed_links 

In [16]:
jobs_filt = filter_links(jobs)

In [ ]:
def scroll(webdriver):
    
    # scroll to bottom of page
    element=webdriver.find_element_by_xpath('//*[@id="resultsCol"]/div[18]')
    return element.location_once_scrolled_into_view

In [34]:
def smooth_scrolling():

    driver= setup_webdriver()
    
    # get specifics 
    driver.get("https://www.indeed.com/")
    driver.find_element_by_id('what').clear()
    time.sleep(1)
    driver.find_element_by_id('what').send_keys('python developer')
    time.sleep(1)
    driver.find_element_by_id('where').clear()
    time.sleep(1)
    driver.find_element_by_id('where').send_keys('New York, NY')
    time.sleep(1)
    driver.find_element_by_id('fj').click()
    time.sleep(1)
    
    driver.find_element_by_id('prime-popover-x').click()
    time.sleep(1)
    
    # scrape jobs from that url 
    job_links = scrape_job_links(driver.current_url)
    # filter to only indeed jobs
    jobs_filt = filter_links(job_links)
    
    # start at a good xpath and count up div tags that appear in every page to 'smooth' scroll
    count = 0
    for i in range(2):
        for j in jobs_filt:
            driver.get(j)
            time.sleep(2)
            count+=1
            #print(count, j)
            str_ = str(count)

            #concat_div = '//*[@id="resultsCol"]/div[' + str_ + ']'
            try: 
                concat_div = '//*[@id="job-content"]/tbody/tr/td[1]/table/tbody/tr/td/div[2]/div[2]/div[' + str_ + ']'
                #concat_div = '//*[@id="job-content"]/tbody/tr/td[1]/table/tbody/tr/td/div[2]/div[' + str_ + ']'
                element=driver.find_element_by_xpath(concat_div)
                element.location_once_scrolled_into_view
                time.sleep(2)
                driver.get(j)
                print(j)
            except:
                browser.get(j)
                print(j)

In [35]:
smooth_scrolling()

https://www.indeed.com/company/NYTP/jobs/Python-Engineer-8578a5b36c166821?fccid=ccff84ac3dd19bb9
https://www.indeed.com/company/RAPS-consulting/jobs/Senior-Python-Developer-21d5e8cd39ab722f?fccid=9b7a3860892e2fcd
https://www.indeed.com/company/Kasisto/jobs/Front-End-Software-Engineer-e8ee0083bfbe3831?fccid=521ca7ccc49e32d5
https://www.indeed.com/company/Chase-Dream-LLC/jobs/Software-Engineer-Python-98563184e467ab84?fccid=ed0eadb761b45e00
https://www.indeed.com/company/LIS-Solutions/jobs/Python-Developer-a424fc091c993864?fccid=58aef85eed32eea8
https://www.indeed.com/company/BlindData/jobs/Software-Engineer-010d7c76f5d1fa19?fccid=55962a85574fa94f
https://www.indeed.com/company/NYTP/jobs/Python-Engineer-8578a5b36c166821?fccid=ccff84ac3dd19bb9
https://www.indeed.com/company/RAPS-consulting/jobs/Senior-Python-Developer-21d5e8cd39ab722f?fccid=9b7a3860892e2fcd
https://www.indeed.com/company/Kasisto/jobs/Front-End-Software-Engineer-e8ee0083bfbe3831?fccid=521ca7ccc49e32d5
https://www.indeed.com

In [89]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [81]:
driver=setup_webdriver()
driver.get(jobs[0])


lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True